# ISRO Moon - Mapping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/ISRO/lte

/content/drive/MyDrive/ISRO/lte


## Importing libraries

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
import requests
from bs4 import BeautifulSoup
import glob
import zipfile
import os
from PIL import Image
from numpy import asarray
import pandas as pd
import sys
from tqdm import tqdm

We have trained our model on images generated by downsampling OHRC images. So for dataset we need to path to folder containing .img files of OHRC images

## Required Data Preprocessing

1.   Loading .img files
2.   Croping them into 256x256 small imgages
3.   Removing black images from the folder 
4.   Converting them into 3 channel RGB images
5.   Bicubic downsampling images (implicitily performed inside the  model)



### Loading and cropping images 

Give path of folder which contains OHRC images in PDS4 format, also provide size of images in which we want to cut .img files

In [ ]:
path_for_dataset = 'path_of_dataset'
size = 256

In [ ]:
cropped_images = 'path_to_folder_where_to_save_cropped_images'
files = glob.glob(path_for_dataset + 'ch2_ohr*')
folders=[]


for file in files:
    if os.path.isdir(file):
        folders.append(file)
        print(file)

for folder in folders:
    print(folder[len(path_for_dataset)+1:])
    path='{}/data/calibrated/{}/{}.img'.format(folder, folder[len(path_for_dataset)+13:len(path_for_dataset)+21], folder[len(path_for_dataset)+1:])
    dtype = np.dtype('B') # unsigned byte
    tree = ET.parse('{}/data/calibrated/{}/{}.xml'.format(folder,folder[len(path_for_dataset)+13:len(path_for_dataset)+21],folder[len(path_for_dataset)+1:]))
    print(tree.getroot())
    ns = {'d': 'http://pds.nasa.gov/pds4/pds/v1','isda':'https://isda.issdc.gov.in/pds4/isda/v1'}
    root = tree.getroot()
    result=root.findall("d:File_Area_Observational",ns)
    shape=[]
    for fao in result:
        result=fao.findall("d:Array_2D_Image/d:Axis_Array/d:elements",ns)
        print(result[0].text,result[1].text)
        shape.append(int(result[0].text))
        shape.append(int(result[1].text))
        print(shape)
    
    #Reading.
    fid = open(path, 'rb')
    data = np.fromfile(fid, dtype)
    image = data.reshape(shape)
    print(data.shape)
    print(image.dtype)


    #Displaying and cutting.
    N=shape[0]//size
    M=shape[1]//size
    for i in range(0,N):
        for j in range(0,M):
            im = Image.fromarray(image[i*size:i*size+size,j*size:j*size+size],mode='L')
            im.save(cropped_images + '/{}_{}.png'.format(i,j))

### Removing black images from the dataset

In [ ]:
sum_list = []
count = 1
filenames = [filename for filename in glob.glob(cropped_images + '/*.png')]
for filename in filenames:
  img = Image.open(filename)
  numpyarray = asarray(img)
  sum = np.sum(numpyarray)
  sum_list.append(sum)

sum_list.sort()
length = len(sum_list)
threshold = sum_list[length/10]

for filename in filenames:
  img = Image.open(filename)
  numpyarray = asarray(img)
  sum = np.sum(numpyarray)
  if(sum<threshold):
    os.remove(filename)

### Converting cropped images to RGB images

In [ ]:
hq_folder_path = cropped_images

filenames = [filename for filename in glob.glob(hq_folder_path +'/*.png')]
for filename in filenames:
  image = cv2.imread(filename)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray_three = cv2.merge([gray,gray,gray])  
  name, ext = path.splitext(filename.split('/')[-1])
  img = Image.fromarray(gray_three)
  img.save(cropped_images + name +'_rgb.png')
  os.remove(filename)

## Training model

Setting up conda environment

In [4]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2023-02-08 18:00:33--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2023-02-08 18:00:33--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh.1’

     0K .......... .......... .......... .......... ..........  0% 13.5M 4s
    50K .......... .......... .......... .......... ..........  

In [5]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ld_impl_linux-64-2.38      |       h1181459_1         732 KB
    libffi-3.3                 |       he6710b0_2          54 KB
    pysocks-1.7.1              |   py36h06a4308_0          30 KB
    six-1.16.0                 |     pyhd3eb1b0_1          19 KB
    conda-package-handling-1.7.3|   py36h27cfd23_1         946 KB
    ncurses-6.3                |       h7f8727e_2         1.0 MB
    openssl-1.1.1s             |       h7f8727e_0         3.8 MB
    cryptography-35.0.0        |   py36hd23ed53_0         1.5 MB
    pip-21.2.2                 |   py36h06a4308_0         2.1 MB
    pyopenssl-22.0.0           |     pyhd3eb1b0_0          49 KB
    urllib3-1.26.8             |     pyhd3eb1b0_0  

ld_impl_linux-64-2.3 |  732 KB | ########## | 100% 
libffi-3.3           |   54 KB | ########## | 100% 
pysocks-1.7.1        |   30 KB | ########## | 100% 
six-1.16.0           |   19 KB | ########## | 100% 
conda-package-handli |  946 KB | ########## | 100% 
ncurses-6.3          |  1.0 MB | ########## | 100% 
openssl-1.1.1s       |  3.8 MB | ########## | 100% 
cryptography-35.0.0  |  1.5 MB | ########## | 100% 
pip-21.2.2           |  2.1 MB | ########## | 100% 
pyopenssl-22.0.0     |   49 KB | ########## | 100% 
urllib3-1.26.8       |  100 KB | ########## | 100% 
python-3.6.13        | 32.5 MB | ########## | 100% 
conda-4.10.3         |  3.1 MB | ########## | 100% 
zlib-1.2.12          |  130 KB | ########## | 100% 
pycparser-2.21       |   94 KB | ########## | 100% 
readline-8.1.2       |  423 KB | ########## | 100% 
tqdm-4.63.0          |   80 KB | ########## | 100% 
idna-3.3             |   55 KB | ########## | 100% 
setuptools-58.0.4    |  979 KB | ########## | 100% 
pycosat-0.6.

In [6]:
!conda --version
!python --version

conda 4.10.3
Python 3.6.13 :: Anaconda, Inc.


In [9]:
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

In [10]:
%cd /content/drive/MyDrive/ISRO/lte

/content/drive/MyDrive/ISRO/lte


In [11]:
%%bash
conda env update -n base --file environment.yaml

Solving environment: ...working... done

freetype-2.11.0      | 618 KB    | ########## | 100% 
bzip2-1.0.8          | 78 KB     | ########## | 100% 
gmp-6.2.1            | 539 KB    | ########## | 100% 
mkl_random-1.1.1     | 327 KB    | ########## | 100% 
ca-certificates-2022 | 118 KB    | ########## | 100% 
pytorch-mutex-1.0    | 3 KB      | ########## | 100% 
libidn2-2.3.2        | 81 KB     | ########## | 100% 
jpeg-9d              | 232 KB    | ########## | 100% 
olefile-0.46         | 48 KB     | ########## | 100% 
openjpeg-2.4.0       | 331 KB    | ########## | 100% 
zstd-1.4.9           | 480 KB    | ########## | 100% 
libiconv-1.15        | 721 KB    | ########## | 100% 
libstdcxx-ng-9.3.0   | 3.1 MB    | ########## | 100% 
openh264-2.1.1       | 711 KB    | ########## | 100% 
nettle-3.7.3         | 809 KB    | ########## | 100% 
typing_extensions-3. | 31 KB     | ########## | 100% 
mkl_fft-1.3.0        | 170 KB    | ########## | 100% 
lame-3.100           | 323 KB    | ######



==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda




For training EDSR + LTE run below cell and for training SwinIR + LTE run next to next cell

In [ ]:
!python train.py --config configs/train-div2k/train_edsr-baseline-lte.yaml --gpu 0

In [ ]:
!python train.py --config configs/train-div2k/train_swinir-lte.yaml --gpu 0

## Test for an image

Run below cell for generating the inferenced output

In [12]:
!python demo.py --input /content/drive/MyDrive/ISRO/lr_test_16x/2765.png --model /content/drive/MyDrive/ISRO/lte/save/_train_edsr-baseline-lte/epoch-last.pth --scale 1 --output output_edsr.png --gpu 0

/usr/local/lib/python3.6/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811805959/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [13]:
!python demo.py --input /content/drive/MyDrive/ISRO/lr_test_16x/2765.png --model /content/drive/MyDrive/ISRO/lte/save/_train_swinir-lte/epoch-last.pth --scale 1 --output output_swinir.png --gpu 0

/usr/local/lib/python3.6/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811805959/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
